In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.gaussian_process.kernels import *
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

In [2]:
dftrain_x = pd.read_csv(r'C:\Users\sedri\Desktop\ETHZ\Master\3_HS22\Advanced Machine Learning\Projects\1_project\X_train.csv')
dftrain_y = pd.read_csv(r'C:\Users\sedri\Desktop\ETHZ\Master\3_HS22\Advanced Machine Learning\Projects\1_project\y_train.csv')
dftest_x = pd.read_csv(r'C:\Users\sedri\Desktop\ETHZ\Master\3_HS22\Advanced Machine Learning\Projects\1_project\X_test.csv')

In [3]:
#drop id 
dftrain_x.pop('id')
dftrain_y.pop('id')
dftest_x.pop('id')

0        0.0
1        1.0
2        2.0
3        3.0
4        4.0
       ...  
771    771.0
772    772.0
773    773.0
774    774.0
775    775.0
Name: id, Length: 776, dtype: float64

In [4]:
#imputation train
#model
imp = IterativeImputer(max_iter=40, random_state=0, n_nearest_features=100)
imp.fit(dftrain_x)
A = imp.transform(dftrain_x)
column_names = dftrain_x.columns
new_datax = pd.DataFrame(A)
new_datax.columns = column_names 

In [5]:
#imputation test
#model
imp = IterativeImputer(max_iter=40, random_state=0, n_nearest_features=100)
imp.fit(dftest_x)
A = imp.transform(dftest_x)
column_names = dftest_x.columns
dftest_x = pd.DataFrame(A)
dftest_x.columns = column_names 

In [6]:
#feature selection train
selector = SelectKBest(f_regression, k=75)
new_datax_reduced = selector.fit_transform(new_datax, np.array(dftrain_y).ravel())
cols = selector.get_support(indices=True)

new_dftest_x = np.array(dftest_x.iloc[:,cols])

C:\Users\sedri\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [7]:
#scale 
scaler = preprocessing.StandardScaler().fit(new_datax_reduced)
X_scaled = scaler.transform(new_datax_reduced)

scaler = preprocessing.StandardScaler().fit(new_dftest_x)
X_test_scaled = scaler.transform(new_dftest_x)

In [ ]:
#train model - Gaussian process
#split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, dftrain_y, test_size=0.2, random_state=42)
kernel = RationalQuadratic() #Matern(nu=0.5) + DotProduct() + WhiteKernel()
GP_model = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=5, normalize_y=True).fit(X_train, y_train)

y_pred = []
y_std = []

for i in range(len(X_test)):
    mu, std = GP_model.predict(X_test[i].reshape(1, -1), return_std=True)
    y_pred.append(mu)
    y_std.append(std)
    
score = r2_score(y_test, np.array(y_pred).squeeze())
print("Gaussian Processes results: {}".format(score))

In [29]:
#train model - Gaussian process - final
#split
X_train = X_scaled
y_train = dftrain_y
kernel = Matern(nu=0.5) + DotProduct() + WhiteKernel()
GP_model = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=5, normalize_y=True).fit(X_train, y_train)

In [30]:
#test - Gaussoian process
y_pred_test = []
y_std_test = []
for i in range(len(new_dftest_x)):
    mu, std = GP_model.predict(X_test_scaled[i].reshape(1, -1), return_std=True)
    y_pred_test.append(mu)
    y_std_test.append(std)

In [31]:
#save solution
solution = []
indeces = []
u = 0
for i in y_pred_test:
    solution.append(i[0][0])
    indeces.append(float(u))
    u+=1
    
df_solution = pd.DataFrame()
df_solution['id'] = indeces
df_solution['y'] = solution

In [11]:
df_solution

,id,y
0,0.0,63.736589
1,1.0,72.571069
2,2.0,67.315190
3,3.0,77.010111
4,4.0,77.339483
...,...,...
771,771.0,67.289574
772,772.0,72.070758
773,773.0,75.507419
774,774.0,67.911701


In [34]:
df_solution.to_csv(index=False, path_or_buf=r'C:\Users\sedri\Desktop\ETHZ\Master\3_HS22\Advanced Machine Learning\Projects\1_project\solution.csv')